# Implement Torus Toy Example Chapter 2.4

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Define System's Dynamics in Euclidean Plane and on Torus

In [ ]:
A = np.array([[1,1], [1,0]])

In [ ]:
def phi(x: np.array) -> np.array:
    x_new = np.dot(A, x) % 1
    return x_new

## Compute Eigendecomposition

In [ ]:
# get eigenvalues and right eigenvectors, i.e. transposed eigenvectors
eig_vals, eig_vects = np.linalg.eig(A)
# retrieve eigenvectors from right eigenvectors
eig_vects = np.transpose(eig_vects)

In [ ]:
print(f"Eigenvalues: {eig_vals}")

In [ ]:
print(f"Eigenvectors: {eig_vects}")

## Analysis of Hyperbolicity and Invertibility

In [ ]:
def is_hyperbolic(eig_vals: np.array) -> bool:
    contr_seg = any(np.abs(eig_val) < 1 for eig_val in eig_vals)
    exp_seg = any(np.abs(eig_val) > 1 for eig_val in eig_vals)

    return contr_seg and exp_seg

In [ ]:
system_hyperbolic = is_hyperbolic(eig_vals)
print(f"Dynamic system is hyperbolic: {system_hyperbolic}")

In [ ]:
def is_invertible(A: np.array) -> bool:
    return np.abs(np.linalg.det(A)) == 1

In [ ]:
system_invertible = is_invertible(A)
print(f"Dynamic system is invertble: {system_invertible}")

## Computation of Bases for Contracting and Expanding Segments 

In [ ]:
def expanding_segments(eig_vals: np.array, eig_vects: np.array) -> np.array:
    exp_eig_spaces_msk = np.abs(eig_vals) > 1
    print(exp_eig_spaces_msk)
    return eig_vects[exp_eig_spaces_msk]

In [ ]:
def contracting_segments(eig_vals: np.array, eig_vects: np.array) -> np.array:
    exp_eig_spaces_msk = np.abs(eig_vals) < 1
    print(exp_eig_spaces_msk)
    return eig_vects[exp_eig_spaces_msk]

In [ ]:
contr_segs = contracting_segments(eig_vals, eig_vects)
print(f"Contracting segments: {contr_segs}")

In [ ]:
exp_segs = expanding_segments(eig_vals, eig_vects)
print(f"Expanding segments: {exp_segs}")

## Translate between Euclidea Plane and Torus

In [ ]:
def q(x: np.array) -> np.array:
    return x % 1

### Show Commutativity of Plane and Torus by q-Translation

In [ ]:
# sample point in R^2 between -10 and 10
x = np.random.uniform(low=-10, high=10, size=2)

In [ ]:
# apply system's dynamics A in plane, then push down to torus by q
plane_dynamics_result = q(np.dot(A, x))
print(f"Next point on torus: {plane_dynamics_result}")

In [ ]:
# push down point to torus, then apply system's dynamics phi on torus
torus_dynamics_result = phi(q(x))
print(f"Next point on torus: {torus_dynamics_result}")

In [ ]:
if np.all(plane_dynamics_result == torus_dynamics_result):
    print(f"System dynamics in plane and on torus commute by q!")
else:
    print(f"System dynamics in plane and on torus do not commute by q!")

## Calculate Inetrsections between Lines

In [ ]:
def perp(a) :
    b = np.empty_like(a)
    b[0] = -a[1]
    b[1] = a[0]
    return b

# line segment a given by endpoints a1, a2
# line segment b given by endpoints b1, b2
def seg_intersect(a, b) :
    a1, a2 = a
    b1, b2 = b

    da = a2 - a1
    db = b2 - b1
    dp = a1 - b1
    dap = perp(da)
    denom = np.dot(dap, db)
    num = np.dot(dap, dp)

    return (num / denom.astype(float))*db + b1

## Construct Markov Partition over Trous for Toy Example

In [ ]:
eig_vals_ratio = np.abs(eig_vals[0] / eig_vals[1])
print(f"Ratio of first to second eigenvalue {eig_vals_ratio}")

In [ ]:
v1 = eig_vects[0]
v2 = eig_vects[1]

In [ ]:
u1 = np.array([0,0])
u2 = np.array([0,1])
u3 = np.array([1,1])
u4 = np.array([1,0])

In [ ]:
lu1 = np.array([[0,0], [0,1]])
lu2 = np.array([[0,1], [1,1]])
lu3 = np.array([[1,1], [1,0]])
lu4 = np.array([[1,0], [0,0]])

In [ ]:
p1 = u1 + (1/v1[0]) * v1
p2 = u2 - (1/v2[1]) * v2
p3 = u3 - (1/v1[0]) * v1
p4 = u4 + (1/v2[1]) * v2

In [ ]:
l_00 = np.array([u1, p1])
l_01 = np.array([u2, p2])
l_10 = np.array([u4, p4])
l_11 = np.array([u3, p3])

In [ ]:
P1a = seg_intersect(l_00, l_01)
P1b = seg_intersect(l_01, l_11)

P3a = seg_intersect(l_10, l_00)

symm_helper_x = seg_intersect(lu3, l_00)
symm_helper_y = q(symm_helper_x)
symm_helper_l00 = np.array([symm_helper_x, symm_helper_y])

p1_symm = symm_helper_y + v1
l_00_symm_extension = np.array([symm_helper_y, p1_symm])
P3b = seg_intersect(l_00_symm_extension, l_01)
l_00_symm_extension = np.array([symm_helper_y, P3b])

In [ ]:
l_00 = np.array([u1, p1])
l_01 = np.array([u2, P1a])
l_10 = np.array([u4, P3a])
l_11 = np.array([u3, P1b])

In [ ]:
# plot unit square
plt.plot(lu1[:, 0], lu1[:, 1], "r-")
plt.plot(lu2[:, 0], lu2[:, 1], "r-")
plt.plot(lu3[:, 0], lu3[:, 1], "r-")
plt.plot(lu4[:, 0], lu4[:, 1], "r-")

# step 1: plot l_00 in expanding direction 
plt.plot(l_00[:, 0], l_00[:, 1], "b-")

# step 2: plot l_01 and l_10 in contracting directions
plt.plot(l_01[:, 0], l_01[:, 1], "b-")
plt.plot(l_10[:, 0], l_10[:, 1], "b-")

# step 3: plot l_11 in expanding direction
plt.plot(l_11[:, 0], l_11[:, 1], "b-")

# step 4: plot symmetric extension of l_00 line
plt.plot(symm_helper_l00[:, 0], symm_helper_l00[:, 1], "m--")
plt.plot(l_00_symm_extension[:, 0], l_00_symm_extension[:, 1], "b-")

# plot intersection points
plt.plot(P1a[0], P1a[1], "bo")
plt.plot(P1b[0], P1b[1], "ro")
plt.plot(P3a[0], P3a[1], "go")
plt.plot(P3b[0], P3b[1], "yo")

plt.show()

In [ ]:
phi(P3b)

## Map Markov Partition over Torus to Euclidean Plane

In [ ]:
P1c = u4 + l_00_symm_extension[1] + u1
P2a = u3 + P3a - u4
P3c = P3a - u4 + u1

In [ ]:
# glue together the splitted rectangles over the unit square by identifying edges with each other
P1 = np.array([P1a, P1b, u3, P1c, P1a])
P2 = np.array([P1b, u2, P2a, u3, P1b])
P3 = np.array([u1, P1a, P3b, P3c, u1])

P = [P1, P2, P3]

In [ ]:
for Pi in P:
    for k in range(Pi.shape[0]-1):
        plt.plot(Pi[k:k+2, 0], Pi[k:k+2, 1], "b-")

plt.show()

## Check Intersection Property

In [ ]:
P1_phi = np.array([np.dot(A,P1a), np.dot(A,P1b), np.dot(A,u3), np.dot(A,P3b)+np.array([1,1]), np.dot(A,P1a)]) - np.array([1,0])

In [ ]:
for Pi in P:
    for k in range(Pi.shape[0]-1):
        plt.plot(Pi[k:k+2, 0], Pi[k:k+2, 1], "b-")


plt.fill(P1_phi[:,0], P1_phi[:,1], "gray", alpha=0.3)

plt.show()

In [ ]:
P2_phi = np.array([np.dot(A, P1b)-np.array([1,0]), np.dot(A,u1), np.dot(A,P3a), np.dot(A,u4)])
P3_phi = np.array([np.dot(A, u1)+np.array([0,1]), np.dot(A, P3a)-np.array([1,0]), np.dot(A, P3b)+np.array([0,1]), np.dot(A, P1a)+np.array([0,1]), np.dot(A, u1)+np.array([0,1])])

In [ ]:
for Pi in P:
    for k in range(Pi.shape[0]-1):
        plt.plot(Pi[k:k+2, 0], Pi[k:k+2, 1], "b-")


plt.fill(P2_phi[:,0], P2_phi[:,1], "gray", alpha=0.3)
plt.fill(P3_phi[:, 0], P3_phi[:, 1], "gray", alpha=0.3)

plt.show()

## Repeat Construction and Start in Second Unstable Branch

In [ ]:
u1 = np.array([0,0])
u2 = np.array([0,1])
u3 = np.array([1,1])
u4 = np.array([1,0])

In [ ]:
lu1 = np.array([[0,0], [0,1]])
lu2 = np.array([[0,1], [1,1]])
lu3 = np.array([[1,1], [1,0]])
lu4 = np.array([[1,0], [0,0]])

In [ ]:
p1 = u1 + (1/v1[0]) * v1
p2 = u2 - (1/v2[1]) * v2
p3 = u3 - (1/v1[0]) * v1
p4 = u4 + (1/v2[1]) * v2

In [ ]:
l_00 = np.array([u1, p1])
l_01 = np.array([u2, p2])
l_10 = np.array([u4, p4])
l_11 = np.array([u3, p3])

In [ ]:
P1a = seg_intersect(l_11, l_01)
P1b = seg_intersect(l_10, l_00)

P3a = seg_intersect(l_10, l_11)

symm_helper_x = seg_intersect(lu1, l_11)
symm_helper_y = symm_helper_x + np.array([1,0])
symm_helper_l11 = np.array([symm_helper_x, symm_helper_y])

p1_symm = symm_helper_y + v1
l_11_symm_extension = np.array([symm_helper_y, p1_symm])
P3b = seg_intersect(l_11_symm_extension, l_10)
l_11_symm_extension = np.array([symm_helper_y, P3b])

In [ ]:
l_00 = np.array([u1, P1b])
l_01 = np.array([u2, P1a])
l_10 = np.array([u4, P3a])
l_11 = np.array([u3, p3])

In [ ]:
# plot unit square
plt.plot(lu1[:, 0], lu1[:, 1], "r-")
plt.plot(lu2[:, 0], lu2[:, 1], "r-")
plt.plot(lu3[:, 0], lu3[:, 1], "r-")
plt.plot(lu4[:, 0], lu4[:, 1], "r-")

# step 1: plot l_11 in expanding direction
plt.plot(l_11[:, 0], l_11[:, 1], "b-")

# step 2: plot l_01 and l_10 in contracting directions
plt.plot(l_01[:, 0], l_01[:, 1], "b-")
plt.plot(l_10[:, 0], l_10[:, 1], "b-")

# step 3: plot l_00 in expanding direction 
plt.plot(l_00[:, 0], l_00[:, 1], "b-")

# step 4: plot symmetric extension of l_11 line
plt.plot(symm_helper_l11[:, 0], symm_helper_l11[:, 1], "m--")
plt.plot(l_11_symm_extension[:, 0], l_11_symm_extension[:, 1], "b-")

# plot intersection points
plt.plot(P1a[0], P1a[1], "bo")
plt.plot(P1b[0], P1b[1], "ro")
plt.plot(P3a[0], P3a[1], "go")
plt.plot(P3b[0], P3b[1], "yo")

plt.show()

## Map Mirrored Markov Partition over Torus to Euclidean Plane

In [ ]:
P1c = u1 - l_00_symm_extension[1]
P2a = u3 + P3a - u4
P3c = P3a - u4 + u1

In [ ]:
# glue together the splitted rectangles over the unit square by identifying edges with each other
P1 = np.array([P3a, P1b, u1, P3b - u4, P3a])
P2 = np.array([u1, P1b, u4, P1a - u2, u1])
P3 = np.array([P3a, u3, u4+P1a, P3b, P3a])

P = [P1, P2, P3]

In [ ]:
for Pi in P:
    for k in range(Pi.shape[0]-1):
        plt.plot(Pi[k:k+2, 0], Pi[k:k+2, 1], "b-")

plt.show()

## Check Intersection Property

In [ ]:
P1_phi = np.array([np.dot(A,P3a), np.dot(A,P1b), np.dot(A,u1), np.dot(A, P3b) - np.array([1,1]), np.dot(A,P3a)])

In [ ]:
for Pi in P:
    for k in range(Pi.shape[0]-1):
        plt.plot(Pi[k:k+2, 0], Pi[k:k+2, 1], "b-")

plt.fill(P1_phi[:,0], P1_phi[:,1], "gray", alpha=0.3)

plt.show()

In [ ]:
P2_phi = np.array([np.dot(A,P1a)-np.array([1,0]), np.dot(A,u2)-np.array([1,0]), np.dot(A,P1b), np.dot(A, u4), np.dot(A,P1a)-np.array([1,0])])
P3_phi = np.array([np.dot(A, P3b)-np.array([1,1]), np.dot(A,P3a)-np.array([1,1]), np.dot(A,u2), np.dot(A,P1a), np.dot(A,P3b)-np.array([1,1])])

In [ ]:
for Pi in P:
    for k in range(Pi.shape[0]-1):
        plt.plot(Pi[k:k+2, 0], Pi[k:k+2, 1], "b-")

plt.fill(P2_phi[:,0], P2_phi[:,1], "gray", alpha=0.3)
plt.fill(P3_phi[:,0], P3_phi[:,1], "gray", alpha=0.3)

plt.show()

## Start Construction with Unstable Branch

In [ ]:
u1 = np.array([0,0])
u2 = np.array([0,1])
u3 = np.array([1,1])
u4 = np.array([1,0])

In [ ]:
lu1 = np.array([[0,0], [0,1]])
lu2 = np.array([[0,1], [1,1]])
lu3 = np.array([[1,1], [1,0]])
lu4 = np.array([[1,0], [0,0]])

In [ ]:
p1 = u1 + (1/v1[0]) * v1
p2 = u2 - (1/v2[1]) * v2
p3 = u3 - (1/v1[0]) * v1
p4 = u4 + (1/v2[1]) * v2

In [ ]:
l_00 = np.array([u1, p1])
l_01 = np.array([u2, p2])
l_10 = np.array([u4, p4])
l_11 = np.array([u3, p3])

In [ ]:
P1a = seg_intersect(l_00, l_01)
P1b = seg_intersect(l_10, l_00)

P3a = seg_intersect(l_10, l_11)

symm_helper_x = seg_intersect(lu2, l_10)
symm_helper_y = symm_helper_x - np.array([0,1])
symm_helper_l11 = np.array([symm_helper_x, symm_helper_y])

p1_symm = symm_helper_y + v2
l_10_symm_extension = np.array([symm_helper_y, p1_symm])
P3b = seg_intersect(l_10_symm_extension, l_00)
l_10_symm_extension = np.array([symm_helper_y, P3b])

In [ ]:
l_00 = np.array([u1, P1b])
l_01 = np.array([u2, P1a])
l_10 = np.array([u4, p4])
l_11 = np.array([u3, P3a])

In [ ]:
# plot unit square
plt.plot(lu1[:, 0], lu1[:, 1], "r-")
plt.plot(lu2[:, 0], lu2[:, 1], "r-")
plt.plot(lu3[:, 0], lu3[:, 1], "r-")
plt.plot(lu4[:, 0], lu4[:, 1], "r-")

# step 1: plot l_10 in contracting direction
plt.plot(l_10[:, 0], l_10[:, 1], "b-")

# step 2: plot l_00 and l_11 in expanding directions
plt.plot(l_00[:, 0], l_00[:, 1], "b-")
plt.plot(l_11[:, 0], l_11[:, 1], "b-")

# step 3: plot l_01 in contracting direction 
plt.plot(l_01[:, 0], l_01[:, 1], "b-")

# step 4: plot symmetric extension of l_01 line
plt.plot(symm_helper_l11[:, 0], symm_helper_l11[:, 1], "m--")
plt.plot(l_10_symm_extension[:, 0], l_10_symm_extension[:, 1], "b-")

# plot intersection points
plt.plot(P1a[0], P1a[1], "bo")
plt.plot(P1b[0], P1b[1], "ro")
plt.plot(P3a[0], P3a[1], "go")
plt.plot(P3b[0], P3b[1], "yo")

plt.show()

## Map Markov Partition over Torus to Euclidean Plane

In [ ]:
# glue together the splitted rectangles over the unit square by identifying edges with each other
P1 = np.array([u2, P3b+np.array([0,1]), P1b, P1a, u2])
P2 = np.array([P3a, u3, P1a+np.array([1,0]), u4, P3a])
P3 = np.array([P3a, P3b+np.array([0,1]), P1b+np.array([0,1]), u3, P3a])

P = [P1, P2, P3]

In [ ]:
for Pi in P:
    for k in range(Pi.shape[0]-1):
        plt.plot(Pi[k:k+2, 0], Pi[k:k+2, 1], "b-")

plt.show()

## Check Intersection Property

In [ ]:
P1_phi = np.array([phi(P1a), u4, phi(P3b)+np.array([1,0]), phi(P1b)+np.array([1,0]), phi(P1a)])
P2_phi = np.array([phi(u1)+np.array([1,1]), phi(P3a), phi(u3)+np.array([0,1]), phi(P1a)+np.array([0,1]), phi(u1)+np.array([1,1])])
P3_phi = np.array([phi(P3b), phi(P1b)+np.array([1,0]), phi(u4)+np.array([1,1]), phi(P3a), phi(P3b)])

In [ ]:
for Pi in P:
    for k in range(Pi.shape[0]-1):
        plt.plot(Pi[k:k+2, 0], Pi[k:k+2, 1], "b-")

plt.fill(P1_phi[:,0], P1_phi[:,1], "gray", alpha=0.3)
plt.fill(P2_phi[:,0], P2_phi[:,1], "gray", alpha=0.3)

plt.show()

In [ ]:
for Pi in P:
    for k in range(Pi.shape[0]-1):
        plt.plot(Pi[k:k+2, 0], Pi[k:k+2, 1], "b-")

plt.fill(P3_phi[:,0], P3_phi[:,1], "gray", alpha=0.3)

plt.show()

## Estimate Transition Probability Kernel

In [ ]:
from shapely.geometry import Polygon, MultiPolygon, Point
from typing import List, Optional

In [ ]:
P1A = Polygon([P1a, u1, u2])
P1B = Polygon([P3a, u3, u4])
P1 = MultiPolygon([P1A, P1B])

In [ ]:
P2A = Polygon([P3b, u1, symm_helper_l11[1]])
P2B = Polygon([symm_helper_l11[0], u2, P1a, P1b])
P2 = MultiPolygon([P2A, P2B])

In [ ]:
P3A = Polygon([P1b, u4, symm_helper_l11[1], P3b])
P3B = Polygon([symm_helper_l11[0], u3, P3a])
P3 = MultiPolygon([P3A, P3B])

In [ ]:
partition = [P1, P2, P3]

In [ ]:
MultiPolygon([P1A, P1B, P2A, P2B, P3A, P3B])

## Monte Carlo Algorithm 5

In [ ]:
def estimate_probability_matrix_algo5(phi: callable, partition: List[MultiPolygon], c: int=10, tau: float=0.001):
    n = len(partition)
    P = np.zeros((n, n))
    P_old = np.ones((n, n))
    C = np.zeros((n, n))

    for i in range(n):
        samples = 0
        while np.max(np.abs(P[i, :] - P_old[i, :])) >= tau:
            p = sample_uniform_random_point(partition[i])
            k = get_subset_index_of_point(Point(phi(p)), partition)
            assert k is not None, f"Error: Cannot find respective subset of {p}"
            C[i,k] = C[i,k] + 1
            samples = samples + 1
            if samples % c == 0:
                P_old[i, :] = P[i, :]
                P[i, :] = C[i, :] / np.linalg.norm(C[i, :], ord=1)

    return P

In [ ]:
def sample_uniform_random_point(area: MultiPolygon):
    min_x, min_y, max_x, max_y = area.bounds
    point = Point([np.random.uniform(min_x, max_x), np.random.uniform(min_y, max_y)])
    while not point.intersects(area):
        point = Point([np.random.uniform(min_x, max_x), np.random.uniform(min_y, max_y)])

    return point

In [ ]:
def get_subset_index_of_point(point: Point, partition: List[MultiPolygon]) -> Optional[int]:
    n = len(partition)
    for k in range(n):
        if point.intersects(partition[k]):
            return k
    return None

In [ ]:
estimate_probability_matrix_algo5(phi, partition, c=1000, tau=0.001)